# Identify bradycardia-liable drugs and their gene interactions (part 1/2 of pharmacovigilance analysis)
## Author: Finn Benned Hansen

In [2]:
import pickle
import pandas as pd
import collections, math,requests

In [3]:
#Load FAERS files
indi = pd.read_pickle(r'J:\BMI\CardiacProteomics\Project data\Cancer medicine link to Atrial Fibrillation (JY + FBH)\FAERS analysis\Johan_final_dump\FAERS\Data\indi_12Q4_19Q2.pkl')
reac = pd.read_pickle(r'J:\BMI\CardiacProteomics\Project data\Cancer medicine link to Atrial Fibrillation (JY + FBH)\FAERS analysis\Johan_final_dump\FAERS\Data\reac_12Q4_19Q2.pkl')
drug = pd.read_pickle(r'J:\BMI\CardiacProteomics\Project data\Cancer medicine link to Atrial Fibrillation (JY + FBH)\FAERS analysis\Johan_final_dump\FAERS\Data\drug_12Q4_19Q2.pkl')
print(len(drug),len(reac),len(indi))

30770692 25435858 20672766


In [4]:
#Extract generic drug names listed as adverse event primary source
drug_PS_id = drug[(drug['role_cod']=='PS')].prod_ai.unique().tolist()
len(drug_PS_id)

4599

In [ ]:
#Make dictonary with  drugname/primary ids and save as pickle file
table =[]
for i in drug_PS_id:
        table.append((i,drug[(drug['role_cod']=='PS') & (drug.prod_ai == i)].primaryid.unique().tolist()))

dict_table = dict(table)

with open(r'O:\CardiacProteomics\Project data\Cancer medicine link to Atrial Fibrillation Functional FollowUp\FBH analyses\dict_table.pkl', 'wb') as f:
    pickle.dump(dict_table,f)

In [5]:
#reopen pickle file
with open(r'J:\BMI\CardiacProteomics\Project data\Cancer medicine link to Atrial Fibrillation Functional FollowUp\FBH analyses\dict_table.pkl', 'rb') as f:
    dict_table = pickle.load(f)

len(dict_table.keys())

4599

In [6]:
drug[drug.primaryid.isin(dict_table[list(dict_table.keys())[1]])]

,primaryid,caseid,drug_seq,role_cod,drugname,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,...,rechal,lot_nbr,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,lot_num,prod_ai
4998647,100035243,10003524,1,PS,LETAIRIS,1,UNKNOWN,"5 MG, QD",NaN,NaN,...,D,NaN,NaN,22081,5,MG,TABLET,QD,1211121A,AMBRISENTAN
4998648,100035243,10003524,2,SS,LETAIRIS,1,NaN,NaN,NaN,NaN,...,D,NaN,NaN,22081,NaN,NaN,TABLET,NaN,NaN,AMBRISENTAN
4998649,100035243,10003524,3,C,ADCIRCA,1,NaN,NaN,NaN,NaN,...,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TADALAFIL
4999240,100071942,10007194,1,PS,LETAIRIS,1,ORAL,"5 MG, QD",NaN,NaN,...,D,NaN,NaN,22081,5,MG,TABLET,QD,NaN,AMBRISENTAN
4999241,100071942,10007194,2,SS,LETAIRIS,1,NaN,NaN,NaN,NaN,...,D,NaN,NaN,22081,NaN,NaN,TABLET,NaN,NaN,AMBRISENTAN
4999359,100092432,10009243,1,PS,LETAIRIS,1,UNKNOWN,"10 MG, UNK",NaN,NaN,...,D,NaN,NaN,22081,10,MG,TABLET,NaN,NaN,AMBRISENTAN
4999360,100092432,10009243,2,SS,LETAIRIS,1,ORAL,"5 MG, UNK",NaN,NaN,...,D,NaN,NaN,22081,5,MG,TABLET,NaN,NaN,AMBRISENTAN
4999371,100096653,10009665,1,PS,LETAIRIS,1,UNKNOWN,"10 MG, QD",NaN,NaN,...,D,NaN,NaN,22081,10,MG,TABLET,QD,1305824A,AMBRISENTAN
4999372,100096653,10009665,2,C,TYVASO,1,NaN,NaN,NaN,NaN,...,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TREPROSTINIL
4999373,100096653,10009665,3,C,ADCIRCA,1,NaN,NaN,NaN,NaN,...,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TADALAFIL


In [7]:
#make df of drugs listed in dictionary with more than 500 AE records, and calculate their bradycardia_AE/total_AE ratios
data1 = []
ai_names = sorted([i for i in dict_table.keys() if type(i)!= float and len(dict_table[i])>500])
for i in ai_names[:]:
    prim_id = dict_table[i]
    r1 = reac[reac.primaryid.isin(prim_id)]
    reacs = list(set([j for j in r1.pt.unique().tolist() if 'bradycardia' in j.lower()]))
    #print(reacs)
    r = r1[r1.pt.isin(reacs)]
    CA_prim_id = r.primaryid.unique().tolist()
    CA_number = len(CA_prim_id)
    CA_reacs = r.pt.unique().tolist()
    data1.append({'drug':i, 'AE_numbers':len(prim_id),'CA_numbers':CA_number, 'CA/AE %': CA_number/len(prim_id)*100})
df_all = pd.DataFrame(data1)
df_all.sort_values(by = 'CA/AE %').reset_index(drop=True)

,AE_numbers,CA/AE %,CA_numbers,drug
0,1103,0.000000,0,(1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR V...
1,699,0.000000,0,CALCIUM CARBONATE\FAMOTIDINE\MAGNESIUM HYDROXIDE
2,733,0.000000,0,FOSAPREPITANT DIMEGLUMINE
3,537,0.000000,0,PENICILLIN G POTASSIUM
4,948,0.000000,0,CANAGLIFLOZIN\METFORMIN HYDROCHLORIDE
5,582,0.000000,0,FORMOTEROL FUMARATE\GLYCOPYRROLATE
6,1000,0.000000,0,DEXTROMETHORPHAN HYDROBROMIDE
7,3898,0.000000,0,FORMOTEROL FUMARATE DIHYDRATE\MOMETASONE FUROATE
8,564,0.000000,0,SODIUM FLUORIDE\TRICLOSAN
9,693,0.000000,0,CANNABIDIOL


In [8]:
#Make ratio list
ratio = df_all['CA/AE %'].tolist()
ratio

[0.0,
 0.09348706762231225,
 0.0,
 0.09363295880149813,
 0.0,
 0.07921702910586972,
 0.0,
 0.0880639479745292,
 0.04161464835622139,
 0.3395052026028732,
 0.0,
 0.20202020202020202,
 0.18393623543838136,
 0.15403573629081946,
 0.07627765064836003,
 0.9681881051175657,
 1.9762845849802373,
 0.12674271229404308,
 0.0,
 1.6954678839256603,
 0.06304609981593924,
 0.0,
 0.0,
 0.0,
 0.0,
 0.11465260261407935,
 0.04151100041511,
 0.9024252679075015,
 0.01059658789869662,
 0.21486892995272885,
 0.03870634784104593,
 0.05954743946010321,
 1.9080659150043366,
 3.1964593066142117,
 0.6737588652482269,
 0.18105009052504525,
 0.5730659025787965,
 0.014225762856533182,
 2.1505376344086025,
 0.490449148167269,
 0.9658052727747325,
 0.500277932184547,
 1.5949493271307527,
 0.19047619047619047,
 0.08286720530350114,
 6.660638939119952,
 6.192299031749606,
 1.9059720457433291,
 3.211755139969799,
 0.37359900373599003,
 0.81799591002045,
 0.14114326040931546,
 0.16769144773616546,
 0.27889680818097307,
 

In [10]:
#Calculate mean ratio plus 3 times the standard deviation of bootstrapped mean
import random, statistics
boot_ratio = statistics.mean(ratio) + 3*statistics.stdev([statistics.mean(random.choices(ratio,k=len(ratio))) for i in range(100000)])
boot_ratio

0.7793397103801759

In [12]:
#make  df as above but with bradycardia/total_AE ratios >= ratio determined by above bootstrapping method 
data = []
ai_names = sorted([i for i in dict_table.keys() if type(i)!= float and len(dict_table[i])>500])
for i in ai_names[:]:
    prim_id = dict_table[i]
    r1 = reac[reac.primaryid.isin(prim_id)]
    reacs = list(set([j for j in r1.pt.unique().tolist() if 'bradycardia' in j.lower()]))
    #print(reacs)
    r = r1[r1.pt.isin(reacs)]
    CA_prim_id = r.primaryid.unique().tolist()
    CA_number = len(CA_prim_id)
    CA_reacs = r.pt.unique().tolist()
    #print(i,CA_reacs,len(prim_id))
    if CA_number/len(prim_id)>=boot_ratio/100:
        indi_treats =indi[indi.primaryid.isin(CA_prim_id) & indi.indi_drug_seq == 1].indi_pt.unique().tolist()
        data.append({'drug':i, 'AE_numbers':len(prim_id),'CA_numbers':CA_number, 'CA/AE %': CA_number/len(prim_id)*100, 'AEs' : set(CA_reacs),
                    'treatment' : indi_treats})
df_CA_low = pd.DataFrame(data)
df_CA_low[df_CA_low['CA/AE %']>=boot_ratio].sort_values(by = 'CA/AE %', ascending= False).reset_index(drop=True)

,AE_numbers,AEs,CA/AE %,CA_numbers,drug,treatment
0,1506,"{Bradycardia, Bradycardia foetal, Sinus bradyc...",17.131474,258,DIGOXIN,"[Arrhythmia, Atrial fibrillation, Product used..."
1,836,"{Bradycardia, Sinus bradycardia}",16.387560,137,BISOPROLOL FUMARATE,"[Hypertension, Product used for unknown indica..."
2,3392,"{Bradycardia, Bradycardia foetal, Bradycardia ...",15.949292,541,BISOPROLOL,"[Hypertension, Suicide attempt, Product used f..."
3,654,"{Bradycardia, Sinus bradycardia}",15.596330,102,NEBIVOLOL,"[Hypertension, Post procedural hypothyroidism,..."
4,543,"{Bradycardia, Sinus bradycardia}",14.364641,78,IVABRADINE,"[Product used for unknown indication, Psoriasi..."
5,724,"{Bradycardia, Sinus bradycardia}",13.950276,101,SUGAMMADEX,"[Prophylaxis, Blood pressure decreased, Analge..."
6,3498,"{Bradycardia, Bradycardia neonatal, Sinus brad...",12.835906,449,ATENOLOL,"[Intentional self-injury, Product used for unk..."
7,564,"{Bradycardia, Sinus bradycardia}",12.234043,69,PROPRANOLOL\PROPRANOLOL HYDROCHLORIDE,"[Product used for unknown indication, Hyperten..."
8,2143,"{Bradycardia, Sinus bradycardia}",12.179188,261,DONEPEZIL,"[Dementia Alzheimer's type, Product used for u..."
9,1245,"{Bradycardia, Sinus bradycardia}",10.923695,136,DILTIAZEM,"[Hypertension, Glaucoma, Blood pressure increa..."


In [13]:
#Prepare cleaned drug name list from above df
x = df_CA_low.drug.tolist()
y = [j for j in [i.split('\\')[0] for i in x if '\\' in i] + [i.split('\\')[1] for i in x if '\\' in i] + x if '\\' not in j]
y_sp = [i.split(' ')[0] for i in y if ' ' in i]
genes = sorted(list(set(y_sp + y)))
genes

['ACETAMINOPHEN',
 'ACYCLOVIR',
 'AGALSIDASE',
 'AGALSIDASE BETA',
 'ALECTINIB',
 'ALECTINIB HYDROCHLORIDE',
 'ALEMTUZUMAB',
 'ALISKIREN',
 'ALPRAZOLAM',
 'ALTEPLASE',
 'AMIODARONE',
 'AMIODARONE HYDROCHLORIDE',
 'AMITRIPTYLINE',
 'AMLODIPINE',
 'AMLODIPINE BESYLATE',
 'ASCORBIC',
 'ASCORBIC ACID',
 'ATENOLOL',
 'AXICABTAGENE',
 'AXICABTAGENE CILOLEUCEL',
 'BACLOFEN',
 'BEDAQUILINE',
 'BEDAQUILINE FUMARATE',
 'BETAMETHASONE',
 'BISOPROLOL',
 'BISOPROLOL FUMARATE',
 'BRIMONIDINE',
 'BRIMONIDINE TARTRATE',
 'BRINZOLAMIDE',
 'BUPIVACAINE',
 'BUPIVACAINE HYDROCHLORIDE',
 'BUPROPION',
 'CANDESARTAN',
 'CANDESARTAN CILEXETIL',
 'CARBAMAZEPINE',
 'CARVEDILOL',
 'CEFAZOLIN',
 'CEFUROXIME',
 'CITALOPRAM',
 'CITALOPRAM HYDROBROMIDE',
 'CLARITHROMYCIN',
 'CLONIDINE',
 'CLONIDINE HYDROCHLORIDE',
 'COAGULATION',
 'COAGULATION FACTOR IX HUMAN',
 'COAGULATION FACTOR VII HUMAN',
 'CRIZOTINIB',
 'CYCLOBENZAPRINE',
 'DARUNAVIR',
 'DARUNAVIR ETHANOLATE',
 'DECITABINE',
 'DESLORATADINE',
 'DEXMEDETOMIDINE

In [14]:
#Prepare new df with gene interaction partners to above cleaned drug list by querying Drug Gene Interaction database
gene = ''
for j in genes:
    gene += j + ','
gene = gene[:-1]

url = 'https://dgidb.org/api/v2/interactions.json?drugs=' + gene
r = requests.get(url)

mt = r.json()['matchedTerms']
length = 0 
res = []
for i in mt:
    res.append((i['drugName'],[j['geneName'] for j in i['interactions']]))
    length += len([j['geneName'] for j in i['interactions']])
print(length)
df = pd.DataFrame(res, columns = ['Drug name', 'Gene interactions']).sort_values(by = 'Drug name')
df

1941


,Drug name,Gene interactions
0,ACETAMINOPHEN,"[PTGS1, FAAH, PTGS2, TRPV1, SULT1A1, SULT1A3, ..."
1,ACYCLOVIR,"[DNTT, ABL1, SLC15A1, IL10, RECQL]"
2,ALECTINIB,"[ALK, EML4, CDKN2A, RET, FGFR1, NPM1, MET, SMAD4]"
3,ALECTINIB HYDROCHLORIDE,"[ALK, RET]"
4,ALEMTUZUMAB,"[CD52, CXCL12]"
5,ALISKIREN,"[ABCB1, REN]"
6,ALPRAZOLAM,"[GABRG1, GABRA1, GABRR3, GABRB1, BRD3, CXCL8, ..."
7,ALTEPLASE,"[FGA, APRT, DES, PLG, NRAS, SPP1, BDNF, IL1B, ..."
8,AMIODARONE,"[KCNH2, ADRA1A, ABCB1, ADRB1, CYP2C9, CYP3A4, ..."
9,AMIODARONE HYDROCHLORIDE,"[GMNN, KCNH2, ERG, SMPD1, AR]"


In [15]:
#Prepare list of interacting genes and gene/drug dictionary
res1 = []
for ind,rows in df.iterrows():
    res1 += rows['Gene interactions']
print(len(res1),len(set(res1)))

res2 = []
for i in list(set(res1)):
    drg=''
    for ind,rows in df.iterrows():
        if i in rows['Gene interactions']:
            drg += rows['Drug name'] + ', '
    res2.append((i,drg[:-2]))
dic_res2 = dict(res2)
dic_res2

1941 749


{'REEP5': 'CITALOPRAM, FLUOXETINE, PAROXETINE',
 'STK39': 'LOSARTAN',
 'FMO5': 'METFORMIN',
 'PDE4D': 'QUETIAPINE',
 'HRH1': 'AMITRIPTYLINE, DESLORATADINE, DIPHENHYDRAMINE, HALOPERIDOL, OLANZAPINE, QUETIAPINE, TRAZODONE',
 'TAC1': 'BACLOFEN, DIGOXIN, FUROSEMIDE, HALOPERIDOL',
 'RRP1B': 'ATENOLOL',
 'TNFRSF11B': 'EPINEPHRINE',
 'AANAT': 'FLUOXETINE',
 'SLC14A2': 'NIFEDIPINE',
 'GRIN2B': 'HALOPERIDOL, MEMANTINE, MEMANTINE HYDROCHLORIDE, QUETIAPINE',
 'S1PR4': 'FINGOLIMOD HYDROCHLORIDE',
 'GRP': 'FUROSEMIDE, NIFEDIPINE, RANITIDINE',
 'BDNF': 'ALTEPLASE, BACLOFEN, CITALOPRAM, DONEPEZIL, ESCITALOPRAM, FLUOXETINE, HALOPERIDOL, LIDOCAINE, LITHIUM, METHYLPREDNISOLONE, OLANZAPINE, PAROXETINE, PROPRANOLOL, TRAMADOL, VENLAFAXINE',
 'PRKAG1': 'METFORMIN',
 'ASXL1': 'DECITABINE',
 'MET': 'ALECTINIB, CRIZOTINIB',
 'IL11': 'CITALOPRAM, ESCITALOPRAM',
 'VEGFA': 'ENALAPRIL, PHENYTOIN',
 'SLC12A2': 'FUROSEMIDE',
 'CPA6': 'METFORMIN',
 'NUCB2': 'HYDROCHLOROTHIAZIDE',
 'NR1I2': 'CARBAMAZEPINE, CEFUROXIME,

In [16]:
def comp(l_compare, l_target):
    if len([i for i in l_compare if i in l_target])>0:
        return True
    else:
        return False

In [17]:
def compar(l_compare, l_target):
    return [dic_res2[i] for i in l_compare if i in l_target]

In [18]:
#load significantly regulated proteins from our richest human heart proteome
df_test = pd.read_excel(r'J:\BMI\CardiacProteomics\Project data\Conduction system heart failure (SM+MS+KK+AS+ET)\Analysis\FAERS\Old stuff\CS_HF_SN_all_signif_proteins_human_orthologs.xlsx')
df_test

,logFC,q mod,Protein IDs,Protein names,Gene names,Human gene names
0,-1.495374,0.024237,O35387,HCLS1-associated protein X-1,Hax1,HAX1
1,-1.461779,0.035024,Q7TSF0;Q61495;Q7TSF1,Desmoglein-1-gamma;Desmoglein-1-alpha;Desmogle...,Dsg1c;Dsg1a;Dsg1b,DSG1
2,-1.337050,0.012424,P70695,"Fructose-1,6-bisphosphatase isozyme 2",Fbp2,FBP2
3,-1.293366,0.041823,Q3UV17,"Keratin, type II cytoskeletal 2 oral",Krt76,KRT76
4,-1.231010,0.040071,Q6P7F1,MAGUK p55 subfamily member 4,Mpp4,MPP4
5,-1.173449,0.037920,Q9CR25,Diphthamide biosynthesis protein 2,Dph2,DPH2
6,-1.150709,0.007220,Q61330,Contactin-2,Cntn2,CNTN2
7,-1.000325,0.026285,P30275,"Creatine kinase U-type, mitochondrial",Ckmt1,CKMT1A
8,-0.951960,0.012424,Q8CIW6,Solute carrier family 26 member 6,Slc26a6,SLC26A6
9,-0.919124,0.043897,Q9WV86,Katanin p60 ATPase-containing subunit A1,Katna1,KATNA1


In [19]:
#Limit proteome df to gene names found above as interaction partner to bradycardia-liable drugs 
df_test['drug hits'] = df_test.apply(lambda row: compar(list(set(res1)), [row['Human gene names']]), axis = 1)

df_test1 = df_test[df_test.apply(lambda row: comp(list(set(res1)), [row['Human gene names']]), axis = 1)].reset_index(drop=True)
df_test1

,logFC,q mod,Protein IDs,Protein names,Gene names,Human gene names,drug hits
0,-0.893219,0.022356,O35219;Q9ER47,Potassium voltage-gated channel subfamily H me...,Kcnh2,KCNH2,"[AMIODARONE, AMIODARONE HYDROCHLORIDE, CITALOP..."
1,-0.727145,0.021843,E9Q557,Desmoplakin,Dsp,DSP,[ENALAPRIL]
2,-0.676680,0.041504,P00520,Tyrosine-protein kinase ABL1,Abl1,ABL1,[ACYCLOVIR]
3,-0.585655,0.018726,O88704;O88703,Potassium/sodium hyperpolarization-activated c...,Hcn1,HCN1,"[IVABRADINE, TRAMADOL]"
4,-0.584619,0.048281,Q00623,Apolipoprotein A-I;Proapolipoprotein A-I;Trunc...,Apoa1,APOA1,[FUROSEMIDE]
5,-0.562440,0.026063,Q9ERZ4,Muscarinic acetylcholine receptor M2,Chrm2,CHRM2,"[AMITRIPTYLINE, OLANZAPINE, ROCURONIUM]"
6,-0.548336,0.028469,P07724,Serum albumin,Alb,ALB,"[DIAZEPAM, DICLOFENAC, OLMESARTAN MEDOXOMIL]"
7,-0.538708,0.029043,Q9ER60,Sodium channel protein type 4 subunit alpha,Scn4a,SCN4A,"[BUPIVACAINE, BUPIVACAINE HYDROCHLORIDE, CARBA..."
8,-0.526779,0.007220,P28652,Calcium/calmodulin-dependent protein kinase ty...,Camk2b,CAMK2B,[NIFEDIPINE]
9,-0.512695,0.025976,O70507;O88705,Potassium/sodium hyperpolarization-activated c...,Hcn4,HCN4,"[IVABRADINE, IVABRADINE HYDROCHLORIDE]"


In [75]:
df_test1.to_excel(r'O:\CardiacProteomics\Project data\Conduction system heart failure (SM+MS+KK+AS+ET)\Analysis\CS_HF_SN_all_signif_proteins_human_orthologs_drughits_bootstrap.xlsx', index=False)